In [50]:
import meshio
import math

In [2]:
import numpy as np
from numpy import pi, sin, cos
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

In [3]:
import pathlib
from pathlib import Path
import sys
import argparse
import os

if sys.version_info.major >= 3 and sys.version_info.minor >= 6:
    import urllib.request as urllib
else:
    import urllib
import tempfile
import re
import tqdm
import json
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/robert/.pyenv/versions/3.9.7/envs/venv/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Download data

In [4]:
# adapted from https://gist.github.com/WaldJohannaU/55f5e35992ea91157b789b15eac4d432

BASE_URL = 'http://campar.in.tum.de/public_datasets/3RScan/'
DATA_URL = BASE_URL + 'Dataset/'
TOS_URL = 'http://campar.in.tum.de/public_datasets/3RScan/3RScanTOU.pdf'
TEST_FILETYPES = ['mesh.refined.v2.obj', 'mesh.refined.mtl', 'mesh.refined_0.png', 'sequence.zip']
# We only provide semantic annotations for the train and validation scans as well as the for the
# reference scans in the test set.
FILETYPES = TEST_FILETYPES + ['labels.instances.annotated.v2.ply', 'mesh.refined.0.010000.segs.v2.json', 'semseg.v2.json']

RELEASE = 'release_scans.txt'
HIDDEN_RELEASE = 'test_rescans.txt'

RELEASE_SIZE = '~94GB'
id_reg = re.compile(r"[a-z0-9]{8}-[a-z0-9]{4}-[a-z0-9]{4}-[a-z0-9]{4}-[a-z0-9]{12}")

def get_scans(scan_file):
    scan_lines = urllib.urlopen(scan_file)
    scans = []
    for scan_line in scan_lines:
        scan_line = scan_line.decode('utf8').rstrip('\n')
        match = id_reg.search(scan_line)
        if match:
            scan_id = match.group()
            scans.append(scan_id)
    return scans

def download_release(release_scans, out_dir, file_types):
    print('Downloading 3RScan release to ' + out_dir + '...')
    for scan_id in tqdm.tqdm(release_scans):
        scan_out_dir = os.path.join(out_dir, scan_id)
        download_scan(scan_id, scan_out_dir, file_types)
    print('Downloaded 3RScan release.')

def download_file(url, out_file):
    # print(url)
    out_dir = os.path.dirname(out_file)
    if not os.path.isdir(out_dir):
        os.makedirs(out_dir)
    if not os.path.isfile(out_file):
        # print('\t' + url + ' > ' + out_file)
        fh, out_file_tmp = tempfile.mkstemp(dir=out_dir)
        f = os.fdopen(fh, 'w')
        f.close()
        urllib.urlretrieve(url, out_file_tmp) 
        os.rename(out_file_tmp, out_file)
    else: pass
        # print('WARNING: skipping download of existing file ' + out_file)

def download_scan(scan_id, out_dir, file_types):
    # print('Downloading 3RScan scan ' + scan_id + ' ...')
    if not os.path.isdir(out_dir):
        os.makedirs(out_dir)
    for ft in file_types:
        url = DATA_URL + '/' + scan_id + '/' + ft
        out_file = out_dir + '/' + ft
        download_file(url, out_file)
    # print('Downloaded scan ' + scan_id)
    

### Define file type and scans to download

In [5]:
file_type = 'semseg.v2.json'
file_types = [file_type]
scan_id = None  # None ==> download all files
out_dir = 'all_3rscans'
pathlib.Path(out_dir).mkdir(exist_ok=True)

### Download files

In [5]:
release_scans = get_scans(BASE_URL + RELEASE)
test_scans = get_scans(BASE_URL + HIDDEN_RELEASE)
file_types_test = TEST_FILETYPES;

if file_type not in TEST_FILETYPES:
    file_types_test = []
else:
    file_types_test = [file_type]
if scan_id:  # download single scan
    if scan_id not in release_scans and scan_id not in test_scans:
        print('ERROR: Invalid scan id: ' + scan_id)
    else:
        out_dir = os.path.join(args.out_dir, scan_id)

        if scan_id in release_scans:
            download_scan(scan_id, out_dir, file_types)
        elif scan_id in test_scans:
            download_scan(scan_id, out_dir, file_types_test)
else: # download entire release
    if len(file_types) == len(FILETYPES):
        print('WARNING: You are downloading the entire 3RScan release which requires ' + RELEASE_SIZE + ' of space.')
    else:
        print('WARNING: You are downloading all 3RScan scans of type ' + file_types[0])
    print('Note that existing scan directories will be skipped. Delete partially downloaded directories to re-download.')
    print('***')
    print('Press any key to continue, or CTRL-C to exit.')
    key = input('')
    download_release(release_scans, out_dir, file_types)
    download_release(test_scans, out_dir, file_types_test)

NameError: name 'get_scans' is not defined

## Parse data

In [55]:
semseg_files[0]

PosixPath('all_3rscans/0cac755e-8d6f-2d13-8c6a-c0979ca34a4f/semseg.v2.json')

In [6]:
data_dir = Path('all_3rscans')
# Warning: not all scenes have a semseg.v2.json (1381 vs 1482)
semseg_files = [f / 'semseg.v2.json' for f in data_dir.iterdir() if f.is_dir() and (f / 'semseg.v2.json').exists()]

In [77]:
scenes = []
for fp in semseg_files:
    with open(fp) as f:
        j = json.load(f)
    bboxes = []
    for obj in j['segGroups']:  
        bboxes.append({
            'centroid': obj['obb']['centroid'],
            'axesLengths': obj['obb']['axesLengths'],
            'normalizedAxes': obj['obb']['normalizedAxes'],
            'label': obj['label'],
            'id': fp.parts[1]
        })
    scenes.append(bboxes)

In [108]:
# `scenes` is a list of scenes.
# each scene is a list of objects.
# each object is specified by a bounding box.
scenes[0][0]

{'centroid': [-1.5771138134959835, 1.7649075851651648, -1.097637007712937],
 'axesLengths': [1.5694338937692993, 0.8191460187924052, 0.8655946113532621],
 'normalizedAxes': [0.8626997470855713,
  -0.5057164430618286,
  -1.1229160780375582e-16,
  0,
  -2.220446049250313e-16,
  1,
  -0.5057164430618286,
  -0.8626997470855713,
  -1.915578245105401e-16],
 'label': 'bean bag',
 'id': '0cac755e-8d6f-2d13-8c6a-c0979ca34a4f'}

## Visualize bounding boxes

### Generate mesh of all bounding boxes and export as `mesh.obj`.

In [141]:
obj

{'centroid': [-2.6628329922224987, 5.701635876901911, -1.1925675807319398],
 'axesLengths': [0.7763073302478619, 0.550465035055614, 0.43033136404818667],
 'normalizedAxes': [0.9346343874931335,
  0.35561007261276245,
  7.896129808066253e-17,
  0,
  -2.220446049250313e-16,
  1,
  0.35561007261276245,
  -0.9346343874931335,
  -2.0753052332026146e-16],
 'label': 'tv stand',
 'id': '19eda6f4-55aa-29a0-8893-8eac3a4d8193'}

In [ ]:
[dict(
        x = bbb[0],
        y = bbb[1],
        z = bbb[2] + ccc[2]/2 + 0.1,
        ax = 50,
        ay = 0,
        text = "Point 5",
        arrowhead = 1,
        # xanchor = "left",
        # yanchor = "bottom"
                    )
                                    ]

In [244]:
scene = [s for s in scenes if s[0]['id'] == '19eda6f4-55aa-29a0-8893-8eac3a4d8193'][0]
mesh = o3d.geometry.TriangleMesh.create_coordinate_frame()
objs = [o for o in scene if (o['label'] not in ['wall', 'floor', 'ceiling', 'stairs', 'window'])]
print(len(objs))
# mesh = o3d.geometry.TriangleMesh()
bbox_annotations = []
for idx, obj in enumerate(objs):
    print(obj)
    width, depth, height = obj["axesLengths"]
    print(idx, obj['label'].rjust(10), 'volume:', width * height * depth)
    obb = o3d.geometry.TriangleMesh.create_box(width=width, height=height, depth=depth)
    obb = obb.translate(-np.array([width, height, depth])/2)
    rot_vec_norm = np.array([0., 0., 1.])  # rotate around z-axis.
    ax, ay = obj["normalizedAxes"][:2]
    angle = math.atan2(ay, ax)
    rot_vec = angle * rot_vec_norm
    rot_mat = o3d.geometry.get_rotation_matrix_from_axis_angle(rot_vec)
    obb = obb.rotate(rot_mat)
    obb = obb.translate(obj["centroid"])
    mesh += obb
    bbox_annotations.append(dict(
        x=obj["centroid"][0],
        y=obj["centroid"][1],
        z=obj["centroid"][2] + depth/2 + 0.01,
        text=obj['label'],
        arrowhead=1,
    ))
o3d.io.write_triangle_mesh("mesh.obj", mesh)

35
{'centroid': [-2.6628329922224987, 5.701635876901911, -1.1925675807319398], 'axesLengths': [0.7763073302478619, 0.550465035055614, 0.43033136404818667], 'normalizedAxes': [0.9346343874931335, 0.35561007261276245, 7.896129808066253e-17, 0, -2.220446049250313e-16, 1, 0.35561007261276245, -0.9346343874931335, -2.0753052332026146e-16], 'label': 'tv stand', 'id': '19eda6f4-55aa-29a0-8893-8eac3a4d8193'}
0   tv stand volume: 0.18389351976884133
{'centroid': [-3.6627032457649147, 5.241511450779247, -1.211505078639724], 'axesLengths': [1.0134892178624586, 0.48300992362924067, 0.38766398005578684], 'normalizedAxes': [0.9999433755874634, 0.010640213266015053, 2.3626019509703895e-18, 0, -2.220446049250313e-16, 1, 0.010640213266015053, -0.9999433755874634, -2.220320317797205e-16], 'label': 'bench', 'id': '19eda6f4-55aa-29a0-8893-8eac3a4d8193'}
1      bench volume: 0.1897713454101929
{'centroid': [-0.2731783146346658, 2.362736870670112, -1.041575607845735], 'axesLengths': [0.5900148632255008, 0.8

True

### Plot `mesh.obj`.

In [245]:
#the coords   x, y, z to plot the  triangulation as a Scatter3d trace
def coords_triangulation(vertices, faces):
    tri_vertices = vertices[faces]
    Xe = []
    Ye = []
    Ze = []
    for T in tri_vertices:
        Xe += [T[k%3][0] for k in range(4)] + [ None]
        Ye += [T[k%3][1] for k in range(4)] + [ None]
        Ze += [T[k%3][2] for k in range(4)] + [ None]
    return Xe, Ye, Ze   

msh = meshio.read("mesh.obj") #https://raw.githubusercontent.com/empet/Datasets/master/Meshes/face-mesh.obj
verts = msh.points
middle = np.max(verts, axis=0) + np.min(verts, axis=0)/2
# verts = verts - middle
I, J, K =  msh.cells_dict["triangle"].T
x, y, z = verts.T

fig = make_subplots(rows=1, cols=1, 
                   horizontal_spacing=0.015,
                   specs=[[{'type': 'scene'}]])

Xe, Ye, Ze = coords_triangulation(verts, msh.cells_dict["triangle"])
# fig.add_trace(go.Scatter3d(x=Xe,
#                      y=Ye,
#                      z=Ze,
#                      mode='lines',
#                      name='',
#                      line=dict(color="red" , width=1.5)), 1, 1)
colorscale = [[0, 'rgb(100,100,100)'], 
              [1, 'rgb(250,250,250)']]
fig.add_trace(go.Mesh3d(x=x, y=y, z=z, 
                          i=I, j=J, k=K, 
                          intensity=z, 
                          colorscale =colorscale,
                          showscale=False,
                          lighting=dict(ambient=0.1,
                                        diffuse=1,
                                        fresnel=3,  
                                        specular=0.5, 
                                        roughness=0.05),
                          lightposition=dict(x=100,
                                             y=200,
                                             z=1000)
                       ), 1, 1)
axis_prop =dict(visible=False, autorange=False)

bbb = np.array([-2.6628329922224987, 5.701635876901911, -1.1925675807319398])
ccc = np.array([0.7763073302478619, 0.550465035055614, 0.43033136404818667])

plotly_scenes = dict(xaxis=dict(range=[-11.41, 11.41], **axis_prop),
              yaxis=dict(range=[-11.41, 11.41], **axis_prop),
              zaxis=dict(range=[-14.67, 10.37], **axis_prop),
              camera_eye=dict(x=-1.85, y=-1.85, z=0.65),
              aspectratio=dict(x=1.15, y=1.15, z=1.1),
              annotations = bbox_annotations)

fig.update_layout(title_text="Scene bounding boxes", title_x=0.5, title_y=0.95,
                  font_size=16, font_color="white",
                  width=800, height=400, autosize=False, 
                  margin=dict(t=2, r=2, b=2, l=2),
                  paper_bgcolor='black',
                  scene=plotly_scenes, scene2=plotly_scenes)
fig.show()